In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 32
num_classes = 10
epochs = 50
l = 40
num_filter = 64
compression = 0.4
dropout_rate = 0.2

In [0]:
#!pip install PyDrive
#import os
#from pydrive.drive import GoogleDrive
#from pydrive.auth import GoogleAuth
#gauth = GoogleAuth()
#drive = GoogleDrive(gauth)
#file_list = drive.ListFile(
#    {'q': "'1SooKSw8M4ACbznKjnNrYvJ5wxuqJ-YCk' in parents"}).GetList()
#(x_train, y_train), (x_test, y_test) = drive.GetList()
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()

#from google.colab import files
#uploaded = files.upload()

In [6]:
'''
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1SooKSw8M4ACbznKjnNrYvJ5wxuqJ-YCk' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)


with open(fname, 'r') as f:
  print(f.read())
  '''

'\n!pip install -U -q PyDrive\nimport os\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials\n\n# 1. Authenticate and create the PyDrive client.\nauth.authenticate_user()\ngauth = GoogleAuth()\ngauth.credentials = GoogleCredentials.get_application_default()\ndrive = GoogleDrive(gauth)\n\n# choose a local (colab) directory to store the data.\nlocal_download_path = os.path.expanduser(\'~/data\')\ntry:\n  os.makedirs(local_download_path)\nexcept: pass\n\n# 2. Auto-iterate using the query syntax\n#    https://developers.google.com/drive/v2/web/search-parameters\nfile_list = drive.ListFile(\n    {\'q\': "\'1SooKSw8M4ACbznKjnNrYvJ5wxuqJ-YCk\' in parents"}).GetList()\n\nfor f in file_list:\n  # 3. Create & download by id.\n  print(\'title: %s, id: %s\' % (f[\'title\'], f[\'id\']))\n  fname = os.path.join(local_download_path, f[\'title\'])\n  print(\'downloading to {}\'.format(fname))\n  f

In [7]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 23s 0us/step


In [0]:
'''

datagen = ImageDataGenerator(rotation_range=90)
datagen.fit(x_train)
'''
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 64, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 64, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 64
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [13]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [15]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
21856/50000 [============>.................] - ETA: 5:39 - loss: 1.6419 - acc: 0.3978

50000/50000 [==============================] - 623s 12ms/step - loss: 1.3937 - acc: 0.4933 - val_loss: 1.3966 - val_acc: 0.5648
Epoch 2/50
 3584/50000 [=>............................] - ETA: 8:51 - loss: 1.0402 - acc: 0.6256

49984/50000 [============================>.] - ETA: 0s - loss: 0.9209 - acc: 0.6744

50000/50000 [==============================] - 609s 12ms/step - loss: 0.9208 - acc: 0.6744 - val_loss: 1.5646 - val_acc: 0.6159
Epoch 3/50
10336/50000 [=====>........................] - ETA: 7:34 - loss: 0.7867 - acc: 0.7227

49984/50000 [============================>.] - ETA: 0s - loss: 0.7572 - acc: 0.7333

...

50000/50000 [==============================] - 611s 12ms/step - loss: 0.0676 - acc: 0.9764 - val_loss: 0.5409 - val_acc: 0.8832
Epoch 49/50
10304/50000 [=====>........................] - ETA: 7:35 - loss: 0.0632 - acc: 0.9781

49984/50000 [============================>.] - ETA: 0s - loss: 0.0682 - acc: 0.9762

50000/50000 [==============================] - 610s 12ms/step - loss: 0.0682 - acc: 0.9762 - val_loss: 0.5449 - val_acc: 0.8820
Epoch 50/50
10304/50000 [=====>........................] - ETA: 7:35 - loss: 0.0631 - acc: 0.9789

49984/50000 [============================>.] - ETA: 0s - loss: 0.0652 - acc: 0.9769

50000/50000 [==============================] - 610s 12ms/step - loss: 0.0652 - acc: 0.9769 - val_loss: 0.5655 - val_acc: 0.8813


In [16]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 37s 4ms/step
Test loss: 0.5655317105069757
Test accuracy: 0.8813


In [17]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')

In [19]:
print(x_train.size)

153600000
